In [24]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import folium
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

print("Modules imported.")

Modules imported.


In [25]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

find_headers = My_table.find_all("th")
headings = [th.text.strip() for th in find_headers]
headings

['Postcode', 'Borough', 'Neighbourhood']

In [26]:
table_classes = {"class": ["sortable"]}
wikitables = soup.findAll("table", table_classes)

df_temp = []
for tr in My_table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postcode, borough, neighbourhood = [td.text.strip() for td in tds]
        df_temp.append(', '.join([postcode, borough, neighbourhood]))
        #print(', '.join([postcode, borough, neighbourhood]))
df_temp[0:5]

['M1A, Not assigned, Not assigned',
 'M2A, Not assigned, Not assigned',
 'M3A, North York, Parkwoods',
 'M4A, North York, Victoria Village',
 'M5A, Downtown Toronto, Harbourfront']

In [27]:
mylist_temp = pd.Series(df_temp)
mylist_split = mylist_temp.str.split(",")
mylist = mylist_split.tolist()

df_wiki = pd.DataFrame(mylist, columns=headings)
df_wiki = df_wiki.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_wiki.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [28]:
df_wiki["Flag"] = np.where(df_wiki["Borough"] == "Not assigned", 1,0)
df_wiki["Neighbourhood"] = np.where(df_wiki["Neighbourhood"] == 'Not assigned', df_wiki.Borough, df_wiki.Neighbourhood)
df_na = df_wiki[df_wiki.Flag == 0].reset_index()
df_na.head()

,index,Postcode,Borough,Neighbourhood,Flag
0,2,M3A,North York,Parkwoods,0
1,3,M4A,North York,Victoria Village,0
2,4,M5A,Downtown Toronto,Harbourfront,0
3,5,M5A,Downtown Toronto,Regent Park,0
4,6,M6A,North York,Lawrence Heights,0


In [29]:
df_na = df_na.drop("Flag", axis=1).groupby(["Postcode", "Borough"]).agg(', '.join).reset_index()
df_na.shape

(103, 3)